# House's Price Prediction

#### I am an absolute beginner to machine learning  and i think this notebook will be very simple that people who just start in machine learning can understand what i am doing 

#### i just start machine learning within this 2 week so i will apply all the cocept that i have learn that relate to regression model,data cleaning,feature engineering in this project.

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Load Data from csv file to dataFrame

In [ ]:
df=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

# Exploring data

In [ ]:
k=sorted(list(df.columns))
k


In [ ]:
len(df.columns)

##### as we see there are 80 columns are indepandance variable and 1 dedpandance variable(SalePrice) in our DataFrame if we put all this columns into our model our model will not optimize because some of the columns may not determine the price so we need to remove it that will increase our model's performance

To idetified we which are the columns that we should use to train our model i recommand you to look this NoteBook: https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python

#### after doing this i've seen some solumns that interesting
##### 1.BsmtQual:it is the quality of the basement
##### 2.TotalBsmt: Total square feet of basement area
##### 3.1stFlrSF: First Floor square feet
##### 4.2ndFlrSF: Second floor square feet
##### 5.GrLivArea: Above grade (ground) living area square feet
##### 6.GarageCars:Size of garage in car capacity
##### 7 .KitchenAbvGr: Kitchens above grade
##### 8.TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
##### 9.MiscFeature	: Miscellaneous feature not covered in other categories



In [ ]:
df1=df[["BsmtQual","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","BedroomAbvGr","KitchenAbvGr","YearBuilt","TotRmsAbvGrd","GarageType","GarageCars","MiscFeature","MiscVal","SalePrice"]]
df1

# Data Cleaning and Feature Engineering

### Data Cleaning
- Data cleaning refers to preparing the data for our model by removing or modify the data that not complete or irrelevant to our model
- It help in increasing the accuracy of our model
- Most of the time, almost 80% in modeling process is dedicate to data cleaning 

### Feature Engineering
- it is the way we apply our domain knowledge or business knowledge about the data to remove outlier or erorr data.
ex: if we know that a room in a house is maximum 300 square feet but in the data is more than that so we will remove from the dataframe
- it is also include in data cleaning

In [ ]:
df1.shape[0]

In [ ]:
df1[df1["MiscFeature"].isna()].shape[0]

### As we see there are a small amount of data that has Miscellaneous feature so we can consider it as outlier then remove them from our dataframe the increase our model performance

In [ ]:
df2=df1[df1["MiscFeature"].isna()]
df2

### And after we only keep the house those doesn't have  Miscellaneous feature so MiscFeature and MiscVal doesn't affect our data anymore then we can remove these columns

In [ ]:
df3=df2.drop(["MiscFeature","MiscVal"],axis="columns")

In [ ]:
df3

In [ ]:
df3[df3.TotalBsmtSF==df3["1stFlrSF"]]

## Remove outlier

In [ ]:
df3.SalePrice.describe()

In [ ]:
df3["price_per_sqt"]=df3.SalePrice/df3.GrLivArea

In [ ]:
df3.price_per_sqt.describe()

# Visualization
- it is very importance because as we are human picture get us to understand thing very qucikly 
- For me i think data visualization is like memes 
- "if we need to explain our visualization that mean our visualization is not good enough"

##### To make clear that those columns have a strong affect on the saleprice i am going to visualize those columns with sale price

In [ ]:
def year_price(df):
    plt.scatter(df.YearBuilt,df.price_per_sqt)
    plt.xlabel("YearBuilt")
    plt.ylabel("Price_per_sqft")
    plt.legend()

#### it seem that year built and price has a positive relationship

In [ ]:
    plt.scatter(df3.TotalBsmtSF,df3.price_per_sqt)
    plt.xlabel("Total basement area")
    plt.ylabel("Price_per_sqft")
    plt.rcParams["figure.figsize"] = (15, 10)
    plt.legend()

## As we see there are some outlier so we need to remove it to increase our model performance

In [ ]:
import numpy as np
def remove_out(df):
    remove_data=pd.DataFrame()
    for year,year_df in df.groupby("YearBuilt"):
        m=np.mean(year_df.price_per_sqt)
        s=np.std(year_df.price_per_sqt)
        out=year_df[(year_df.price_per_sqt>(m-s)) & (year_df.price_per_sqt<(m+s))]
        remove_data=pd.concat([remove_data,out],ignore_index=True)
    return remove_data
df4=remove_out(df3)
df4

    
    
    
    
    

In [ ]:
year_price(df4)

In [ ]:
year_price(df3)

In [ ]:
df4.fillna("unknown",inplace=True)

In [ ]:

plt.bar(df4.GarageType,df4.SalePrice)
plt.ylabel("price")
plt.xlabel("GrarageType")

In [ ]:
plt.bar(df4.BsmtQual,df4.SalePrice)
plt.ylabel("price")
plt.xlabel("BsmtQual")

#### so GarageType and BmstQuality have big impact to price so i'm not going to delete it

## clean bad data

In [ ]:
df4[df4.BsmtQual.isna()].shape

In [ ]:
df4[df4.GarageType.isna()].shape

# Building and Testing model

#### remove some columns that we are not going to use it anymore

In [ ]:
df4.columns

In [ ]:
df5=df4.drop(["1stFlrSF","2ndFlrSF","KitchenAbvGr","BedroomAbvGr","KitchenAbvGr","price_per_sqt"],axis="columns")

In [ ]:
df5.isna().sum()

In [ ]:
df5

In [ ]:
from sklearn.preprocessing import LabelEncoder
la=LabelEncoder()
df5["BsmtQual"]=la.fit_transform(df5.BsmtQual)

In [ ]:
df5

In [ ]:
le=LabelEncoder()
df5["GarageType"]=le.fit_transform(df5.GarageType)

In [ ]:
df5

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x=df5.drop("SalePrice",axis="columns")
x

In [ ]:
y=df5.SalePrice

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
y_train

## import all the model that i am going to use

In [ ]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.tree import DecisionTreeRegressor


In [ ]:
model=LinearRegression()
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

### we need to do it more times to make suffle our data to test the score

In [ ]:
from sklearn.model_selection import ShuffleSplit,cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),x_train,y_train,cv=cv)

## Test model using gridsearchcv

In [ ]:
from sklearn.model_selection import GridSearchCV



In [ ]:
models={
    'linear_regeression':{
        'model':LinearRegression(),
        "params":{
            "normalize":[True,False]
        }
    },
    "lasso":{
        "model":Lasso(),
        "params":{
            'alpha':[1,2],
            'selection':["random",'cyclic']
        }
    },
    "decision_tree":{
        'model':DecisionTreeRegressor(),
        'params':{
            'criterion':["mse","friedman_mse"],
            'splitter':['best','random']
        }
    }
}

In [ ]:
def test_score(a,b):
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for model_name,mode in models.items():
        gs=GridSearchCV(mode["model"],mode["params"],cv=cv,return_train_score=False)
        gs.fit(a,b)
        scores.append({
            'model':model_name,
            'best_score':gs.best_score_,
            'best_parms':gs.best_params_
        })
    return pd.DataFrame(scores)
test_score(x_train,y_train)


### so i'm going to chose lasso as my model for prediction the home price

In [ ]:
modell=Lasso(alpha=2,selection="cyclic")
modell.fit(x_train,y_train)

In [ ]:
df5

In [ ]:
modell.predict([[3,1470,3493,1880,10,1,3]])

In [ ]:
df5["price_predict"]=modell.predict(df5.drop("SalePrice",axis="columns"))

In [ ]:
df5